#### Knn Titanic

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

In [25]:
import pandas
import numpy
#from scipy.stats import mode
from sklearn import neighbors
from sklearn.neighbors import DistanceMetric 
from pprint import pprint

T_TRAIN = 'raw_data/train_titanic.csv'
T_TEST = 'raw_data/test_titanic.csv'
titanic_dataframe = pandas.read_csv(T_TRAIN, header=0)
print('length: {0} '.format(len(titanic_dataframe)))
titanic_dataframe.head(5)

length: 891 


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Hypothesis:

Women, children and rich people were more likely to survive.

In [26]:
titanic_dataframe.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
print('dropped')
titanic_dataframe.describe()

dropped


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [27]:
titanic_train_raw = pandas.read_csv(T_TRAIN, header=0)
avg_age_all_raw = titanic_train_raw['Age'].mean()
titanic_train['Age'] = titanic_train_raw['Age'].fillna(avg_age_all_raw)
titanic_dataframe.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Replaced NaN with 'E'. Created a new column 'Port' with integer values that represent the port. Created a new column 'Gender' with integer values that represent the sex. 

In [28]:
titanic_dataframe['Embarked'].unique()
titanic_dataframe['Embarked'] = titanic_dataframe['Embarked'].fillna('E')
titanic_dataframe['Port'] = titanic_dataframe['Embarked'].map({'S':1, 'C':2, 'Q':3, 'E':4}).astype(int)


titanic_dataframe['Sex'].unique()
titanic_dataframe['Gender'] = titanic_dataframe['Sex'].map({'female': 0, 'male': 1}).astype(int)
titanic_dataframe = titanic_dataframe.drop(['Sex', 'Embarked'], axis=1)
print('length: {0} '.format(len(titanic_dataframe)))

length: 891 


In [30]:
test_dataframe = pandas.read_csv(T_TEST, header=0)
test_dataframe.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
avg_age_all_raw = test_dataframe['Age'].mean()
test_dataframe['Age'] = test_dataframe['Age'].fillna(avg_age_all_raw)

test_dataframe['Embarked'].unique()
test_dataframe['Embarked'] = test_dataframe['Embarked'].fillna('E')
test_dataframe['Port'] = test_dataframe['Embarked'].map({'S':1, 'C':2, 'Q':3, 'E':4}).astype(int)


test_dataframe['Sex'].unique()
test_dataframe['Gender'] = test_dataframe['Sex'].map({'female': 0, 'male': 1}).astype(int)
test_dataframe = test_dataframe.drop(['Sex', 'Embarked'], axis=1)
test_dataframe.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Port,Gender
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000,418.000000,418.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188,1.464115,0.636364
std,120.810458,0.841838,12.634534,0.896760,0.981429,55.907576,0.685516,0.481622
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800,1.000000,0.000000
50%,1100.500000,3.000000,30.272590,0.000000,0.000000,14.454200,1.000000,1.000000
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000,2.000000,1.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,3.000000,1.000000


In [32]:
model = neighbors.KNeighborsClassifier()
model.fit(titanic_dataframe.values, test_dataframe.values)

result = numpy.c_[test_data[:,0].astype(int), output.astype(int)]


df_result = pandas.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('titanic.csv', index=False) 

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').